In [163]:
import pickle

In [164]:
import random as rd
def generate_ksat_mat(k, m, p=0.5):
       while True: 
        try:
            matrix = np.zeros((m, k), dtype=int)
            unique_rows = set()
            negation_rows = set()
        
            for i in range(m):
                while True:
                    row = np.zeros(k, dtype=int)
                    for j in range(k):
                        if rd.random() < p:  # Probability of including a variable in a clause
                            # Randomly assign either 1 or -1 (true or negated)
                            row[j] = 1 if rd.random() < 0.5 else -1
        
                    row_tuple = tuple(row)
                    negated_row_tuple = tuple(-x for x in row)  # Create a tuple of negated values
        
                    # Check that the row is not all zeros, not already included, and not a negation of an included row
                    if row_tuple not in unique_rows and np.any(row != 0) and negated_row_tuple not in unique_rows:
                        unique_rows.add(row_tuple)
                        negation_rows.add(negated_row_tuple)  # Keep track of negations to avoid adding them later
                        matrix[i, :] = row
                        break
            solve_ksat_class(matrix.tolist())
            return matrix.tolist()
        except Exception as e:
            print('REEEEEEEEDDOOOOOOOO')

In [165]:
generate_ksat_mat(5,5)

    solver failure.
    solver failure.
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 1, 1, 0, 0]


[[1, 0, 1, 0, 0],
 [0, 1, 0, 0, 1],
 [1, 0, 1, -1, -1],
 [0, 0, 1, 0, 1],
 [-1, 1, 0, -1, 0]]

In [166]:
first_case=[]
for k in range(3,16):
    temp = []
    for _ in range(15):
        temp.append(generate_ksat_mat(k=k,m=k))
    first_case.append(temp)

    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 1, 1]
    solver failure.
    solver failure.
    solver failure.
REEEEEEEEDDOOOOOOOO
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 0, 0]
    solver failure.
    solver failure.
    solver failure.
Classical solution : [1, 0, 0]
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 0, 1]
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 0, 1]
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 1, 0]
    solver failure.
    solver failure.
    solver failure.
REEEEEEEEDDOOOOOOOO
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 0, 1]
    solver failure.
    solver failure.
    solver failure.
Classical solution : [0, 0, 0]
    solver failure.
    solver failure.
    solver failure.
Classical solution : [1, 0, 0]
    solver failure.
 

In [181]:
with open('first_case.pkl', 'wb') as f:
    pickle.dump(first_case, f)

In [55]:
import numpy as np

#problem = [[1,0,0,0,0],[1,0,1,0,0],[0,-1,0,0,0],[0,0,0,-1,0],[0,0,1,1,0],[0,0,0,0,-1],[1,0,0,0,1]]
problem = generate_satisfiable_ksat_matrix(5,5,0.5)

In [56]:
problem

array([[ 0, -1,  0, -1,  0],
       [ 1, -1,  0, -1, -1],
       [ 0,  0,  1,  0,  0],
       [-1,  0, -1,  1,  1],
       [ 0,  0,  0,  0,  0]])

In [57]:
def create_formula(problem):
    string = ''
    for row in problem:
        temp = []
        counter = 1
        for element in row:
            if element == 1:
                temp.append('x' + str(counter))
            elif element == -1:
                temp.append('not x' + str(counter))
            counter += 1

        if len(temp) == 1:
            string += '(' + temp[0] + ')' + 'and'
        elif len(temp) > 1:
            string += '(' + ' or '.join(temp) + ')' + 'and'
    return '(' + string[:-3] + ')'

In [58]:
formula1 = create_formula(problem)

In [59]:
formula1

'((not x2 or not x4)and(x1 or not x2 or not x4 or not x5)and(x3)and(not x1 or not x3 or x4 or x5))'

In [60]:
'''formula = """
    ( (x1) and (x1 or x3) and (not x2) and (not x4) and (x3 or x4) )
"""'''

'formula = """\n    ( (x1) and (x1 or x3) and (not x2) and (not x4) and (x3 or x4) )\n"""'

In [61]:
try:
    import ttg

    print(ttg.Truths(["x1", "x2", "x3", "x4"], [formula1[1:-1]]))
except:
    print("Please import 'truth-table-generator' in order to print the truth table")

Please import 'truth-table-generator' in order to print the truth table


In [62]:
from classiq import RegisterUserInput, construct_grover_model

register_size = RegisterUserInput(size=1)

qmod = construct_grover_model(
    num_reps=1,
    expression="(" + formula1 + ")",
    definitions=[
        ("x1", register_size),
        ("x2", register_size),
        ("x3", register_size),
        ("x4", register_size),
        ("x5", register_size),
    ],
)

In [63]:
from classiq import write_qmod

write_qmod(qmod, "4_sat_grover")

In [64]:
from classiq import QuantumProgram, synthesize, create_model, Preferences, set_preferences

preferences = Preferences(
    backend_service_provider="IBM Quantum", backend_name="ibm_sherbrooke"
)
model = set_preferences(qmod, preferences)
qprog = synthesize(model)

In [65]:
circuit = QuantumProgram.from_qprog(qprog)
#circuit.show()

In [66]:
print(circuit.transpiled_circuit)

outputs={<QuantumFormat.QASM: 'qasm'>: "// G...2];
" (length=198892)} qasm_version=<QasmVersion.V2: '2.0'> depth=1240 count_ops={'rz': 751, 'sx': 562, 'ecr': 507, 'x': 477} logical_to_physical_input_qubit_map=[58, 60, 64, 53, 62, 61, 63, 72, 59, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126] logical_to_physical_output_qubit_map=[64, 60, 72, 61, 59, 62, 58, 53, 63, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5

In [33]:
from classiq import execute, set_quantum_program_execution_preferences
from classiq.execution import (
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
    ExecutionPreferences,
)

backend_preferences = ExecutionPreferences(
    backend_preferences=ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR
    )
)

qprog = set_quantum_program_execution_preferences(qprog, backend_preferences)
optimization_result = execute(qprog).result()

In [34]:
res = optimization_result[0].value

In [40]:
result = res.counts_of_multiple_outputs(("x1", "x2", "x3", "x4","x5"))

In [43]:
result

{('0', '1', '1', '0', '1'): 14,
 ('0', '0', '0', '0', '1'): 21,
 ('0', '1', '0', '1', '0'): 20,
 ('0', '1', '0', '1', '1'): 20,
 ('1', '0', '1', '0', '0'): 15,
 ('1', '0', '0', '1', '0'): 401,
 ('1', '0', '0', '0', '1'): 22,
 ('1', '0', '0', '0', '0'): 17,
 ('0', '1', '1', '0', '0'): 16,
 ('1', '0', '1', '1', '0'): 403,
 ('0', '0', '1', '1', '1'): 429,
 ('1', '0', '1', '0', '1'): 23,
 ('0', '1', '0', '0', '1'): 16,
 ('1', '0', '0', '1', '1'): 19,
 ('0', '0', '0', '0', '0'): 18,
 ('1', '0', '1', '1', '1'): 351,
 ('1', '1', '1', '0', '0'): 12,
 ('1', '1', '0', '0', '0'): 14,
 ('1', '1', '0', '1', '0'): 17,
 ('0', '0', '0', '1', '0'): 20,
 ('0', '0', '1', '1', '0'): 9,
 ('0', '1', '1', '1', '1'): 12,
 ('0', '1', '0', '0', '0'): 16,
 ('0', '0', '0', '1', '1'): 17,
 ('1', '1', '0', '1', '1'): 15,
 ('1', '1', '1', '1', '1'): 10,
 ('1', '1', '0', '0', '1'): 15,
 ('0', '1', '1', '1', '0'): 23,
 ('1', '1', '1', '1', '0'): 13,
 ('0', '0', '1', '0', '0'): 19,
 ('1', '1', '1', '0', '1'): 18,
 ('0'

In [47]:
max_key = max(result, key=result.get)
maxkey = list([int(x) for x in max_key])
maxkey

[0, 0, 1, 1, 1]

In [48]:
import matplotlib.pyplot as plt
from classiq import construct_combinatorial_optimization_model
from classiq.applications.combinatorial_optimization import OptimizerConfig, QAOAConfig
import numpy as np
from qiskit import QuantumCircuit as qc
from pyomo.environ import ConcreteModel, Var, Binary, minimize, Objective, ConstraintList
from pyomo.opt import SolverFactory
from classiq import show, synthesize
from classiq import execute
import pandas as pd
from classiq.applications.combinatorial_optimization import get_optimization_solution_from_pyo
from classiq import set_execution_preferences
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences
from classiq import write_qmod
import random as rd

In [49]:
def solve_ksat_class(matrix):
    model = ConcreteModel()
    n, m = len(matrix[0]), len(matrix)
    model.x = Var(range(n), domain=Binary)
    model.obj = Objective(expr=0, sense=minimize)
    model.constraints = ConstraintList()

    for i in range(m):
        clause_expr = 0
        for j in range(n):
            if matrix[i][j] == 1:
                clause_expr += model.x[j]  # Variable itself
            elif matrix[i][j] == -1:
                clause_expr += (1 - model.x[j])  # Negation of the variable
        # Add at least one true literal per clause
        model.constraints.add(clause_expr >= 1)
        solver = SolverFactory('glpk')
        solver.solve(model, tee=False)
    solarray = [int(model.x[i].value) for i in range(n)]
    print(f"Classical solution : {solarray}")
    return [int(model.x[i].value) for i in range(n)]


In [101]:
solve_ksat_class([[-1,0],[-1,0]])

    solver failure.
    solver failure.


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [51]:
def verify_solution(matrix, solution):
    num_clauses = len(matrix)
    num_vars = len(matrix[0])
    
    for i in range(num_clauses):
        clause_satisfied = False
        for j in range(num_vars):
            if matrix[i][j] != 0:
                # Determine the truth value of the j-th variable in this clause
                literal_truth = (solution[j] == 1) if matrix[i][j] == 1 else (solution[j] == 0)
                if literal_truth:
                    clause_satisfied = True
                    break  # No need to check more literals in this clause
        if not clause_satisfied:
            return False  # If any clause is not satisfied, return False immediately
    
    return True  # All clauses are satisfied

In [52]:
verify_solution(matrix=problem, solution=maxkey)

True